In [1]:
# Import all of the python packages used in this workflow.
import scipy
import numpy as np
from collections import OrderedDict
import os, sys
from pylab import *
import pandas as pd
import numpy as np
import osr
import xarray as xr
import geopandas as gpd
from datetime import datetime
from datetime import timedelta  
import json

# Import CSO gdf (metadata) and df (daily data) 

In [2]:
gdf = gpd.read_file('../CSO_SNOTEL_sites.geojson')
df = pd.read_csv('../CSO_SNOTEL_data_SWEDmeters.csv') 
gdf.head()

,code,longitude,latitude,name,elevation_m,easting,northing,geometry
0,314_WY_SNTL,-110.445442,43.940189,Base Camp,2151.887939453125,544505.845453,4.865379e+06,POINT (-110.44544 43.94019)
1,347_MT_SNTL,-111.128029,44.508320,Black Bear,2490.216064453125,489823.440274,4.928341e+06,POINT (-111.12803 44.50832)
2,350_WY_SNTL,-109.793327,44.376671,Blackwater,2980.944091796875,596129.923439,4.914418e+06,POINT (-109.79333 44.37667)
3,353_WY_SNTL,-110.609734,42.964001,Blind Bull Sum,2636.52001953125,531828.554679,4.756891e+06,POINT (-110.60973 42.96400)
4,379_WY_SNTL,-109.670212,43.697330,Burroughs Creek,2667,607155.527746,4.839116e+06,POINT (-109.67021 43.69733)


# Import baseline .par parameters 

In [3]:
with open('par_base.json') as f:
    base = json.load(f)

base['prec_lapse_rate']

['0.35,0.35,0.35,0.30,0.25,0.20, 0.20,0.20,0.20,0.25,0.30,0.35',
 'prec_lapse_rate (1/km)\n',
 146]

# Function to edit text files 
## Edit snowmodel.par and snowmodel.inc to run SnowModel as line -> original code

In [4]:
#function to edit SnowModel Files other than .par
def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()

# Functions to adjust calibraiton parameters
## Edit snowmodel.par to run SnowModel as line -> Dave's code

In [5]:
parFile = '/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill/snowmodel.par'
incFile = '/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill/code/snowmodel.inc'
compileFile = '/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill/code/compile_snowmodel.script'
ctlFile = '/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill/ctl_files/wo_assim/swed.ctl'
sweFile = '/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill/outputs/wo_assim/swed.gdat'

In [6]:
#Edit the par file to set parameters with new values
def edit_par(par_dict,parameter,new_value):
    lines = open(parFile, 'r').readlines()
    if par_dict[parameter][2] == 14 or par_dict[parameter][2] == 17 \
    or par_dict[parameter][2] == 18 or par_dict[parameter][2] == 19 \
    or par_dict[parameter][2] == 93 or par_dict[parameter][2] == 95 \
    or par_dict[parameter][2] == 97 or par_dict[parameter][2] == 100 \
    or par_dict[parameter][2] == 102 or par_dict[parameter][2] == 104 \
    or par_dict[parameter][2] == 107 or par_dict[parameter][2] == 108:
        text = str(new_value)+'\n'
    else:
        text = str(new_value)+'\t\t\t!'+par_dict[parameter][1]
    lines[par_dict[parameter][2]] = text
    out = open(parFile, 'w')
    out.writelines(lines)
    out.close()

In [7]:
#print(base.keys())
print(base['lapse_rate'])
print(base['prec_lapse_rate'])

['4.4,5.9,7.1,7.8,8.1,8.2,8.1,8.1,7.7,6.8,5.5,4.7', 'lapse_rate - temp lapse rate\n', 144]
['0.35,0.35,0.35,0.30,0.25,0.20, 0.20,0.20,0.20,0.25,0.30,0.35', 'prec_lapse_rate (1/km)\n', 146]


In [8]:
#edit snowmodel.par
edit_par(base,'nx',np.shape(gdf)[0])
edit_par(base,'ny',1)
edit_par(base,'xmn',487200)
edit_par(base,'ymn',4690100)
edit_par(base,'dt',21600)
edit_par(base,'iyear_init',2014)
edit_par(base,'imonth_init',10)
edit_par(base,'iday_init',1)
edit_par(base,'xhour_init',0)
edit_par(base,'max_iter',7300)
edit_par(base,'met_input_fname','met/mm_wy_2014-2019.dat')
edit_par(base,'ascii_topoveg',1)
edit_par(base,'topo_ascii_fname','topo_vege/DEM_WY.asc')
edit_par(base,'veg_ascii_fname','topo_vege/NLCD2016_WY.asc')
edit_par(base,'xlat',40.2)
edit_par(base,'run_snowtran',0)
edit_par(base,'barnes_lg_domain',1)
edit_par(base,'snowmodel_line_flag',1)
edit_par(base,'lapse_rate','.28,1.2,2.8,4.2,4.5,4.4,4.0,3.8,3.7,3.4,2.6,0.87')#
edit_par(base,'prec_lapse_rate','0.4,0.4,0.46,0.41,0.27,0.24,0.21,0.17,0.22,0.32,0.43,0.39')#
edit_par(base,'print_inc',4)
edit_par(base,'print_var_01','y')
edit_par(base,'print_var_09','y')
edit_par(base,'print_var_10','y')
edit_par(base,'print_var_11','y')
edit_par(base,'print_var_12','y')
edit_par(base,'print_var_14','y')
edit_par(base,'print_var_18','y')

##edit snowmodel.inc
replace_line(incFile, 12, '      parameter (nx_max='+str(np.shape(gdf)[0]+1)+',ny_max=2)\n')
#replace_line(incFile, 12, '      parameter (nx_max=1383,ny_max=2477)\n')#full domain


##edit compile_snowmodel.script
#replace_line(compileFile, 16, '#pgf77 -O3 -mcmodel=medium -I$path -o ../snowmodel $path$filename1 $path$filename2 $path$filename3 $path$filename4 $path$filename5 $path$filename6 $path$filename7 $path$filename8 $path$filename9 $path$filename10\n')
#replace_line(compileFile, 20, 'gfortran -O3 -mcmodel=medium -I$path -o ../snowmodel $path$filename1 $path$filename2 $path$filename3 $path$filename4 $path$filename5 $path$filename6 $path$filename7 $path$filename8 $path$filename9 $path$filename10\n')

# Function to compile/run SnowModel and extract relevant forcing parameters

In [9]:
#Compile SnowModel - with Dave's code - should only have to do this once
%cd /nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill/code/
#run compile script 
! ./compile_snowmodel.script
%cd  /nfs/attic/dfh/Aragon2/Notebooks/calibration_python

/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill/code
/nfs/attic/dfh/Aragon2/Notebooks/calibration_python


In [10]:
%%time

def runSnowModel():
    %cd /nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill/
    ! nohup ./snowmodel
    %cd  /nfs/attic/dfh/Aragon2/Notebooks/calibration_python

runSnowModel()

/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill
nohup: ignoring input and appending output to ‘nohup.out’
/nfs/attic/dfh/Aragon2/Notebooks/calibration_python
CPU times: user 11.6 ms, sys: 8.07 ms, total: 19.7 ms
Wall time: 14.3 s


In [11]:
def get_mod_dims():
    #get model data from .ctl file 
    f=open(ctlFile)
    lines=f.readlines()
    nx = int(lines[9].split()[1])
    xll = int(float(lines[9].split()[3]))
    clsz = int(float(lines[9].split()[4]))
    ny = int(lines[10].split()[1])
    yll = int(float(lines[10].split()[3]))
    num_sim_days = int(lines[14].split()[1])
    st = datetime.strptime(lines[14].split()[3][3:], '%d%b%Y').date()
    ed = st + timedelta(days=(num_sim_days-1))
    print('nx=',nx,'ny=',ny,'xll=',xll,'yll=',yll,'clsz=',clsz,'num_sim_days=',num_sim_days,'start',st,'end',ed)
    f.close()
    return nx, ny, xll, yll, clsz, num_sim_days, st, ed

nx, ny, xll, yll, clsz, num_sim_days, st, ed = get_mod_dims()

nx= 30 ny= 1 xll= 487200 yll= 4690100 clsz= 100 num_sim_days= 1825 start 2014-10-01 end 2019-09-29


# Function to convert SnowModel output to numpy array

This function is to be used when running SnowModel as a line

In [12]:
## Build a function to convert the binary model output to numpy array

def get_mod_output_line(inFile,stn):
    #open the grads model output file, 'rb' indicates reading from binary file
    grads_data = open(inFile,'rb')
    # convert to a numpy array 
    numpy_data = np.fromfile(grads_data,dtype='float32',count=-1)
    #close grads file 
    grads_data.close()
    #reshape the data 
    numpy_data = np.reshape(numpy_data,(num_sim_days,ny,nx))
    #swe only at station point
    data = np.squeeze(numpy_data[:,0,stn]) 

    return data

# Function for calculating performance statistics

In [13]:
#compute model performance metrics
def calc_metrics():
    swe_stats = np.zeros((5,np.shape(gdf)[0]))
    for i in range(np.shape(gdf)[0]):
        mod_swe = get_mod_output_line(sweFile,i)
        loc = gdf['code'][i]
        stn_swe = df[loc].values
        
        #remove days with zero SWE at BOTH the station and the SM pixel
        idx = np.where((stn_swe != 0) | (mod_swe != 0))
        mod_swe = mod_swe[idx]
        stn_swe = stn_swe[idx]
        
        #remove days where station has nan values 
        idx = np.where(~np.isnan(stn_swe))
        mod_swe = mod_swe[idx]
        stn_swe = stn_swe[idx]
        
        #mean absolute error
        swe_stats[0,i] = (sum(abs(mod_swe - stn_swe)))/mod_swe.shape[0]

        #mean bias error
        swe_stats[1,i] = (sum(mod_swe - stn_swe))/mod_swe.shape[0]

        #root mean squared error
        swe_stats[2,i] = np.sqrt((sum((mod_swe - stn_swe)**2))/mod_swe.shape[0])

        # Nash-Sutcliffe model efficiency coefficient, 1 = perfect, assumes normal data 
        nse_top = sum((mod_swe - stn_swe)**2)
        nse_bot = sum((stn_swe - mean(stn_swe))**2)
        swe_stats[3,i] = (1-(nse_top/nse_bot))

        # Kling-Gupta Efficiency, 1 = perfect
        kge_std = (np.std(mod_swe)/np.std(stn_swe))
        kge_mean = (mean(mod_swe)/mean(stn_swe))
        kge_r = corrcoef((stn_swe),(mod_swe))[1,0]
        swe_stats[4,i] = (1 - (sqrt((kge_r-1)**2)+((kge_std-1)**2)+(kge_mean-1)**2))
    return swe_stats

swe_stats = calc_metrics()

# Create dictionary of variables to iterate over 

In [72]:
ro_snowmax=arange(float(base ['ro_snowmax'][0])-250,
                  float(base ['ro_snowmax'][0])+250,50)
cf_precip_scalar=arange(float(base ['cf_precip_scalar'][0])-.3,
                        float(base ['cf_precip_scalar'][0])+.3,.1)
ro_adjust=arange(float(base ['ro_adjust'][0])-3,
                 float(base ['ro_adjust'][0])+3,1)
Total_runs = len(ro_snowmax)*len(cf_precip_scalar)*len(ro_adjust)
print('Total number of calibration runs = ',Total_runs)

Total number of calibration runs =  420


# Calibration!

In [107]:
%%time
cal_params = {}
swe_stats = np.empty([Total_runs,5,np.shape(gdf)[0]])
n=0
for i in range(len(ro_snowmax)):
    edit_par(base,'ro_snowmax',ro_snowmax[i])
    for j in range(len(cf_precip_scalar)):
        edit_par(base,'cf_precip_scalar',cf_precip_scalar[j])
        for k in range(len(ro_adjust)):
            #print(n)
            edit_par(base,'ro_adjust',ro_adjust[k])
            cal_params[n] = ['ro_snowmax',ro_snowmax[i],'cf_precip_scalar',
                             cf_precip_scalar[j],'ro_adjust',ro_adjust[k]]
            #nx, ny, xll, yll, clsz, num_sim_days, st, ed = runSnowModel()
            swe_stats[n-1,:,:] = calc_metrics()
            n=n+1
            
print(cal_params.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [ ]:
for key in base:
    print(key)
    print(base[key])

In [108]:
#Turn NDarray into xarray 
calibration_run = np.arange(0,swe_stats.shape[0],1)
metric = ['MAE','MBE','RMSE','NSE','KGE']
station = gdf['code'].values

cailbration = xr.DataArray(
    swe_stats,
    dims=('calibration_run', 'metric', 'station'), 
    coords={'calibration_run': calibration_run, 
            'metric': metric, 'station': station})

In [109]:
cailbration.attrs['long_name']= 'Calibration performance metrics'
cailbration.attrs['standard_name']= 'cal_metrics'

d = OrderedDict()
d['calibration_run'] = ('calibration_run', calibration_run)
d['metric'] = ('metric', metric)
d['station'] = ('station', station)
d['cal_metrics'] = cailbration

ds = xr.Dataset(d)
ds.attrs['description'] = "SnowModel line calibration performance metrics"
ds.attrs['calibration_parameters'] = "ro_snowmax,cf_precip_scalar,ro_adjust"
ds.attrs['model_parameter'] = "SWE [m]"

ds.calibration_run.attrs['standard_name'] = "calibration_run"
ds.calibration_run.attrs['axis'] = "N"

ds.metric.attrs['long_name'] = "calibration_metric"
ds.metric.attrs['axis'] = "metric"

ds.station.attrs['long_name'] = "station_id"
ds.station.attrs['axis'] = "station"
ds

<xarray.Dataset>
Dimensions:          (calibration_run: 420, metric: 5, station: 30)
Coordinates:
  * calibration_run  (calibration_run) int64 0 1 2 3 4 5 ... 415 416 417 418 419
  * metric           (metric) <U4 'MAE' 'MBE' 'RMSE' 'NSE' 'KGE'
  * station          (station) object '314_WY_SNTL' ... '868_WY_SNTL'
Data variables:
    cal_metrics      (calibration_run, metric, station) float64 0.06446 ... 0.3124
Attributes:
    description:             SnowModel line calibration performance metrics
    calibration_parameters:  ro_snowmax,cf_precip_scalar,ro_adjust
    model_parameter:         SWE [m]

In [110]:
#export to netcdf
ds.to_netcdf('calibration_test.nc', format='NETCDF4', engine='netcdf4')
#test
ds_read = xr.open_dataset('calibration_test.nc')
ds_read

<xarray.Dataset>
Dimensions:          (calibration_run: 420, metric: 5, station: 30)
Coordinates:
  * calibration_run  (calibration_run) int64 0 1 2 3 4 5 ... 415 416 417 418 419
  * metric           (metric) object 'MAE' 'MBE' 'RMSE' 'NSE' 'KGE'
  * station          (station) object '314_WY_SNTL' ... '868_WY_SNTL'
Data variables:
    cal_metrics      (calibration_run, metric, station) float64 ...
Attributes:
    description:             SnowModel line calibration performance metrics
    calibration_parameters:  ro_snowmax,cf_precip_scalar,ro_adjust
    model_parameter:         SWE [m]

In [120]:
#save calibration parameters 
import json

json = json.dumps(cal_params)
f = open("cal_test.json","w")
f.write(json)
f.close()